In [2]:
import pandas as pd
import json
import re

In [3]:
with open('data/Neighbourhoods-historical_140-4326.geojson') as f:
    geojson_data = json.load(f)

neighbour_df = pd.json_normalize(geojson_data)
neighbour_df.head()


,type,properties._id,properties.AREA_ID,properties.AREA_ATTR_ID,properties.PARENT_AREA_ID,properties.AREA_SHORT_CODE,properties.AREA_LONG_CODE,properties.AREA_NAME,properties.AREA_DESC,properties.CLASSIFICATION,properties.CLASSIFICATION_CODE,properties.OBJECTID,geometry.type,geometry.coordinates
0,Feature,1,2501286,26022481,0,030,030,Brookhaven-Amesbury (30),Brookhaven-Amesbury (30),Not an NIA or Emerging Neighbourhood,NA,17818369,MultiPolygon,"[[[[-79.5029569066111, 43.6957370912699], [-79..."
1,Feature,2,2501285,26022480,0,106,106,Humewood-Cedarvale (106),Humewood-Cedarvale (106),Not an NIA or Emerging Neighbourhood,NA,17818385,MultiPolygon,"[[[[-79.4184870633337, 43.6836294842459], [-79..."
2,Feature,3,2501284,26022479,0,091,091,Weston-Pelham Park (91),Weston-Pelham Park (91),Neighbourhood Improvement Area,NIA,17818401,MultiPolygon,"[[[[-79.4600458827451, 43.6672259620873], [-79..."
3,Feature,4,2501283,26022478,0,048,048,Hillcrest Village (48),Hillcrest Village (48),Not an NIA or Emerging Neighbourhood,NA,17818417,MultiPolygon,"[[[[-79.3434566072996, 43.7951657083605], [-79..."
4,Feature,5,2501282,26022477,0,129,129,Agincourt North (129),Agincourt North (129),Not an NIA or Emerging Neighbourhood,NA,17818433,MultiPolygon,"[[[[-79.2421274578355, 43.8024720055488], [-79..."


In [4]:
new_df = neighbour_df[["properties.AREA_NAME", "geometry.coordinates"]]
new_df.head(1)


,properties.AREA_NAME,geometry.coordinates
0,Brookhaven-Amesbury (30),"[[[[-79.5029569066111, 43.6957370912699], [-79..."


In [5]:
# Correctly renaming the column
renamed_neighbourhood_df = new_df.rename(columns={'properties.AREA_NAME': 'Neighbourhood Name',
                                                  'geometry.coordinates' : "Co-ordinates"})
renamed_neighbourhood_df.head()

,Neighbourhood Name,Co-ordinates
0,Brookhaven-Amesbury (30),"[[[[-79.5029569066111, 43.6957370912699], [-79..."
1,Humewood-Cedarvale (106),"[[[[-79.4184870633337, 43.6836294842459], [-79..."
2,Weston-Pelham Park (91),"[[[[-79.4600458827451, 43.6672259620873], [-79..."
3,Hillcrest Village (48),"[[[[-79.3434566072996, 43.7951657083605], [-79..."
4,Agincourt North (129),"[[[[-79.2421274578355, 43.8024720055488], [-79..."


In [6]:
renamed_neighbourhood_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Neighbourhood Name  140 non-null    object
 1   Co-ordinates        140 non-null    object
dtypes: object(2)
memory usage: 2.3+ KB


In [7]:
covid_df = pd.read_json("data/COVID19_cases.json")
covid_df.head()

,_id,Assigned_ID,Outbreak Associated,Age Group,Neighbourhood Name,FSA,Source of Infection,Classification,Episode Date,Reported Date,Client Gender,Outcome,Ever Hospitalized,Ever in ICU,Ever Intubated
0,1,1,NO,50 to 59 Years,Willowdale East,M2N,Travel,CONFIRMED,2020-01-22,2020-01-23,FEMALE,RESOLVED,No,No,No
1,2,2,NO,50 to 59 Years,Willowdale East,M2N,Travel,CONFIRMED,2020-01-21,2020-01-23,MALE,RESOLVED,Yes,No,No
2,3,3,NO,20 to 29 Years,Parkwoods-Donalda,M3A,Travel,CONFIRMED,2020-02-05,2020-02-21,FEMALE,RESOLVED,No,No,No
3,4,4,NO,60 to 69 Years,Church-Yonge Corridor,M4W,Travel,CONFIRMED,2020-02-16,2020-02-25,FEMALE,RESOLVED,No,No,No
4,5,5,NO,60 to 69 Years,Church-Yonge Corridor,M4W,Travel,CONFIRMED,2020-02-20,2020-02-26,MALE,RESOLVED,No,No,No


In [8]:
covid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 414830 entries, 0 to 414829
Data columns (total 15 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   _id                  414830 non-null  int64 
 1   Assigned_ID          414830 non-null  int64 
 2   Outbreak Associated  414830 non-null  object
 3   Age Group            414830 non-null  object
 4   Neighbourhood Name   414830 non-null  object
 5   FSA                  414830 non-null  object
 6   Source of Infection  414830 non-null  object
 7   Classification       414830 non-null  object
 8   Episode Date         414830 non-null  object
 9   Reported Date        414830 non-null  object
 10  Client Gender        414830 non-null  object
 11  Outcome              414830 non-null  object
 12  Ever Hospitalized    414830 non-null  object
 13  Ever in ICU          414830 non-null  object
 14  Ever Intubated       414830 non-null  object
dtypes: int64(2), object(13)
memory usa

In [10]:
# Function to remove the code within brackets using regex
def remove_code(name):
    return re.sub(r'\s*\(.*?\)', '', name)

# Apply the function to the 'Neighbourhood Name' column
renamed_neighbourhood_df['Neighbourhood Name'] = renamed_neighbourhood_df['Neighbourhood Name'].apply(remove_code)

# Display the updated DataFrame
renamed_neighbourhood_df.head()

,Neighbourhood Name,Co-ordinates
0,Brookhaven-Amesbury,"[[[[-79.5029569066111, 43.6957370912699], [-79..."
1,Humewood-Cedarvale,"[[[[-79.4184870633337, 43.6836294842459], [-79..."
2,Weston-Pelham Park,"[[[[-79.4600458827451, 43.6672259620873], [-79..."
3,Hillcrest Village,"[[[[-79.3434566072996, 43.7951657083605], [-79..."
4,Agincourt North,"[[[[-79.2421274578355, 43.8024720055488], [-79..."


In [11]:
renamed_neighbourhood_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Neighbourhood Name  140 non-null    object
 1   Co-ordinates        140 non-null    object
dtypes: object(2)
memory usage: 2.3+ KB


In [13]:
merged_df = pd.merge(covid_df,renamed_neighbourhood_df,how='inner',on ='Neighbourhood Name')
merged_df.head()

,_id,Assigned_ID,Outbreak Associated,Age Group,Neighbourhood Name,FSA,Source of Infection,Classification,Episode Date,Reported Date,Client Gender,Outcome,Ever Hospitalized,Ever in ICU,Ever Intubated,Co-ordinates
0,1,1,NO,50 to 59 Years,Willowdale East,M2N,Travel,CONFIRMED,2020-01-22,2020-01-23,FEMALE,RESOLVED,No,No,No,"[[[[-79.4122474447867, 43.7666877822519], [-79..."
1,2,2,NO,50 to 59 Years,Willowdale East,M2N,Travel,CONFIRMED,2020-01-21,2020-01-23,MALE,RESOLVED,Yes,No,No,"[[[[-79.4122474447867, 43.7666877822519], [-79..."
2,3,3,NO,20 to 29 Years,Parkwoods-Donalda,M3A,Travel,CONFIRMED,2020-02-05,2020-02-21,FEMALE,RESOLVED,No,No,No,"[[[[-79.3305547263443, 43.7397898788133], [-79..."
3,4,4,NO,60 to 69 Years,Church-Yonge Corridor,M4W,Travel,CONFIRMED,2020-02-16,2020-02-25,FEMALE,RESOLVED,No,No,No,"[[[[-79.3767169389805, 43.6624188721439], [-79..."
4,5,5,NO,60 to 69 Years,Church-Yonge Corridor,M4W,Travel,CONFIRMED,2020-02-20,2020-02-26,MALE,RESOLVED,No,No,No,"[[[[-79.3767169389805, 43.6624188721439], [-79..."


In [14]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384994 entries, 0 to 384993
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   _id                  384994 non-null  int64 
 1   Assigned_ID          384994 non-null  int64 
 2   Outbreak Associated  384994 non-null  object
 3   Age Group            384994 non-null  object
 4   Neighbourhood Name   384994 non-null  object
 5   FSA                  384994 non-null  object
 6   Source of Infection  384994 non-null  object
 7   Classification       384994 non-null  object
 8   Episode Date         384994 non-null  object
 9   Reported Date        384994 non-null  object
 10  Client Gender        384994 non-null  object
 11  Outcome              384994 non-null  object
 12  Ever Hospitalized    384994 non-null  object
 13  Ever in ICU          384994 non-null  object
 14  Ever Intubated       384994 non-null  object
 15  Co-ordinates         384994 non-nu

In [4]:
# This will make the json file which is 4Gb in size
merged_df.to_json("covid_data_modified.json", orient='records', lines=True)


NameError: name 'merged_df' is not defined

In [3]:
# Convert DataFrame to JSON
json_output = merged_df.to_dict(orient='records')

# Save to JSON file
with open('COVID_Modified.json', 'w') as f:
 
    json.dump(json_output, f, indent=4)

NameError: name 'merged_df' is not defined

In [4]:
# Function to convert your data to GeoJSON
def convert_to_geojson(data):
    features = []
    for record in data:
        feature = {
            "type": "Feature",
            "geometry": {
                "type": "Polygon",
                "coordinates": record["Co-ordinates"]
            },
            "properties": {
                "Age Group": record["Age Group"],
                "Neighbourhood Name": record["Neighbourhood Name"],
                "Classification": record["Classification"],
                "Episode Date": record["Episode Date"],
                "Reported Date": record["Reported Date"],
                "Client Gender": record["Client Gender"],
                "Outcome": record["Outcome"]
            }
        }
        features.append(feature)
    
    geojson = {
        "type": "FeatureCollection",
        "features": features
    }
    
    return geojson

# Read the data from the input file
with open('data/coviddataModified.json', 'r') as f:
    lines = f.readlines()

# Assuming the file contains multiple JSON objects, parse each line as JSON
data = []
for line in lines:
    try:
        data.append(json.loads(line))
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")

# Convert your data to GeoJSON
geojson_data = convert_to_geojson(data)

# Save the GeoJSON data to an output file
with open('COVID_new.geojson', 'w') as f:
    json.dump(geojson_data, f, indent=2)

print("GeoJSON file created successfully.")

GeoJSON file created successfully.
